In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cobra
from cobra.io import read_sbml_model
from load_json import load_json_model

In [2]:
model = read_sbml_model('Pan_putida2-Final.xml')
old_model = load_json_model('iJN1463.json')
model

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


Name,Pan_putida2
Memory address,0x07ff42ed10be0
Number of metabolites,2525
Number of reactions,3301
Number of groups,0
Objective expression,1.0*BiomassKT2440_WT3 - 1.0*BiomassKT2440_WT3_reverse_2cf2b
Compartments,"cytosol, extracellular space, periplasm, h, r, m, w"


In [3]:
modelseed_subsystems = pd.read_table('ModelSEED_Subsystems.tsv')
modelseed_rxns = pd.read_table('reactions.tsv')
subsystems = modelseed_rxns.rename(columns={'id':'Reaction'}).merge(modelseed_subsystems,on='Reaction')
subsystems

,Reaction,abbreviation,name,code,stoichiometry,is_transport,equation,definition,reversibility,direction,...,compound_ids,status,is_obsolete,linked_reaction,notes,source,Class,Sub-class,Name,Role
0,rxn00001,R00004,diphosphate phosphohydrolase,(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd0...,"-1:cpd00001:0:0:""H2O"";-1:cpd00012:0:0:""PPi"";2:...",0,(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd0...,(1) H2O[0] + (1) PPi[0] <=> (2) Phosphate[0] +...,>,=,...,cpd00001;cpd00009;cpd00012;cpd00067,OK,0,rxn27946;rxn27947;rxn27948;rxn32487;rxn38157;r...,GCC|HB|EQC|EQU,Primary Database,Experimental Subsystems,-,271-Bsub,Manganese-dependent inorganic pyrophosphatase ...
1,rxn00001,R00004,diphosphate phosphohydrolase,(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd0...,"-1:cpd00001:0:0:""H2O"";-1:cpd00012:0:0:""PPi"";2:...",0,(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd0...,(1) H2O[0] + (1) PPi[0] <=> (2) Phosphate[0] +...,>,=,...,cpd00001;cpd00009;cpd00012;cpd00067,OK,0,rxn27946;rxn27947;rxn27948;rxn32487;rxn38157;r...,GCC|HB|EQC|EQU,Primary Database,Phosphorus Metabolism,-,Phosphate_metabolism,Inorganic pyrophosphatase (EC 3.6.1.1)
2,rxn00001,R00004,diphosphate phosphohydrolase,(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd0...,"-1:cpd00001:0:0:""H2O"";-1:cpd00012:0:0:""PPi"";2:...",0,(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd0...,(1) H2O[0] + (1) PPi[0] <=> (2) Phosphate[0] +...,>,=,...,cpd00001;cpd00009;cpd00012;cpd00067,OK,0,rxn27946;rxn27947;rxn27948;rxn32487;rxn38157;r...,GCC|HB|EQC|EQU,Primary Database,Phosphorus Metabolism,-,Phosphate_metabolism,Manganese-dependent inorganic pyrophosphatase ...
3,rxn00001,R00004,diphosphate phosphohydrolase,(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd0...,"-1:cpd00001:0:0:""H2O"";-1:cpd00012:0:0:""PPi"";2:...",0,(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd0...,(1) H2O[0] + (1) PPi[0] <=> (2) Phosphate[0] +...,>,=,...,cpd00001;cpd00009;cpd00012;cpd00067,OK,0,rxn27946;rxn27947;rxn27948;rxn32487;rxn38157;r...,GCC|HB|EQC|EQU,Primary Database,Regulation and Cell signaling,-,HPr_catabolite_repression_system,Inorganic pyrophospatase PpaX
4,rxn00002,R00005,urea-1-carboxylate amidohydrolase,(1) cpd00001[0] + (1) cpd00742[0] <=> (2) cpd0...,"-1:cpd00001:0:0:""H2O"";-3:cpd00067:0:0:""H+"";-1:...",0,(1) cpd00001[0] + (3) cpd00067[0] + (1) cpd007...,(1) H2O[0] + (3) H+[0] + (1) Allophanate[0] =>...,>,>,...,cpd00001;cpd00011;cpd00013;cpd00067;cpd00742,OK,0,rxn30346;rxn35525,GCC|EQC|EQU,Primary Database,Experimental Subsystems,Pyrimidines,Novel_non-oxidative_pathway_of_Uracil_catabolism,Allophanate hydrolase (EC 3.5.1.54)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9815,rxn13784,rxn13784,RNA transcription,<=> (1) cpd17043[0],"1:cpd17043:0:0:""RNA transcription""",0,<=> (1) cpd17043[0],<=> (1) RNA transcription[0],?,=,...,cpd17043,CPDFORMERROR,0,NaN,GCP|EQP,Orphan,RNA Metabolism,Transcription,Transcription_factors_cyanobacterial_RpoD-like...,RNA polymerase sigma factor RpoD
9816,rxn13784,rxn13784,RNA transcription,<=> (1) cpd17043[0],"1:cpd17043:0:0:""RNA transcription""",0,<=> (1) cpd17043[0],<=> (1) RNA transcription[0],?,=,...,cpd17043,CPDFORMERROR,0,NaN,GCP|EQP,Orphan,RNA Metabolism,Transcription,"Transcription_initiation,_bacterial_sigma_factors",RNA polymerase sigma factor RpoD
9817,rxn13784,rxn13784,RNA transcription,<=> (1) cpd17043[0],"1:cpd17043:0:0:""RNA transcription""",0,<=> (1) cpd17043[0],<=> (1) RNA transcription[0],?,=,...,cpd17043,CPDFORMERROR,0,NaN,GCP|EQP,Orphan,Stress Response,Oxidative stress,Glutaredoxins,tRNA (cytosine34-2'-O-)-methyltransferase (EC ...
9818,rxn13784,rxn13784,RNA transcription,<=> (1) cpd17043[0],"1:cpd17043:0:0:""RNA transcription""",0,<=> (1) cpd17043[0],<=> (1) RNA transcription[0],?,=,...,cpd17043,CPDFORMERROR,0,NaN,GCP|EQP,Orphan,Virulence,Iron Scavenging Mechanisms,"Heme,_hemin_uptake_and_utilization_systems_in_...",Two-component response regulator SA14-24


In [4]:
subs = list(set([old_model.reactions.get_by_id(j).subsystem for j in [w.id for w in model.reactions if w.id in [k.id for k in old_model.reactions]]]))

In [ ]:
metabolic_categories = {
    "Central Carbon Metabolism": [
        "S_Glycolysis",
        "S_TCA_Cycle", 
        "S_Gluconeogenesis",
        "S_Pentose_Phosphate_Pathway",
        "S_Pyruvate_Metabolism",
        "S_Oxidative_Phosphorylation",
        "S_Glyoxylate_and_dicarboxylate_metabolism"
    ],
    
    "Alternative Carbon Metabolism": [
        "S_Alternate_Carbon__Ribose_Metabolism",
        "S_Alternate_Carbon__Fructose_Metabolism", 
        "S_Alternate_Carbon__Propanoate_Metabolism",
        "S_Alternate_Carbon__Butanediol_Metabolism",
        "S_Alternate_Carbon__Levulinate_Metabolism",
        "S_Alternate_Carbon__Ascorbate_and_Aldarate_Metabolism",
        "S_Alternate_Carbon",
        "S_Starch_and_Sucrose_Metabolism",
        "S_Butanoate_Metabolism",
    ],
    
    "Amino Acid Metabolism": [
        "S_Lysine_Metabolism",
        "S_Arginine_and_Proline_Metabolism",
        "S_Phenylalanine_Tyrosine_Tryptophan_Biosynthesis",
        "S_Alanine_and_Aspartate_Metabolism",
        "S_Glycine__Serine_and_threonine_metabolism", 
        "S_Valine__Leucine__and_Isoleucine_Metabolism",
        "S_Glutamate_Metabolism",
        "S_Methionine_Metabolism",
        "S_Histidine_Metabolism",
        "S_Cysteine_Metabolism",
        "S_Urea_cycleamino_group_metabolism"
    ],
    
    "Nucleotide Metabolism": [
        "S_Nucleotide_Salvage_Pathway",
        "S_Pyrimidine_Metabolism",
        "S_Purine_Metabolism", 
        "S_Alternate_Carbon_and_Nitrogen_source__Nucleotide_Metabolism"
    ],
    
    "Fatty Acid and Lipid Metabolism": [
        "S_Fatty_Acid__Biosynthesis",
        "S_Fatty_Acid_Metabolism",
        "S_Glycerophospholipid_Metabolism",
        "Aliphatic open-chain ketones metabolism"
    ],
    
    "Cell Envelope and Wall Biosynthesis": [
        "S_Cell_Envelope_Biosynthesis",
        "S_Cell_Envelope_Biosynthesis__Cellulose_Metabolism",
        "S_Cell_Envelope_Biosynthesis__Peptidoglycan_Biosynthesis",
        "S_Cell_Envelope_Biosynthesis__Lipid_A_Biosynthesis",
        "S_Cell_Envelope_Biosynthesis__LPS_Biosynthesis",
        "S_Cell_Envelope_Biosynthesis__Biosynthesis_of_L_glycero_D_manno_heptose__Hep_",
        "S_Cell_Envelope_Biosynthesis__O_antigen_Biosynthesis",
        "Murein Recycling"
    ],
    
    "Cofactor and Vitamin Biosynthesis": [
        "S_Cofactor_and_Prosthetic_Group_Biosynthesis",
        "S_Cofactor_and_Prosthetic_Group_Biosynthesis__Ubiquinone_biosynthesis",
        "S_Cofactor_and_Prosthetic_Group_Biosynthesis__Nicotinamide_Biosynthesis",
        "S_Cofactor_and_Prosthetic_Group_Biosynthesis__Vitamin_B6_Metabolism",
        "S_Cofactor_and_Prosthetic_Group_Biosynthesis__Terpenoid_backbone_biosynthesis",
        "S_Cofactor_and_Prosthetic_Group_Biosynthesis__FeS_clusters_metabolism",
        "S_Cofactor_and_Prosthetic_Group_Biosynthesis__Molybdopterin_Biosynthesis",
        "S_Cofactor_and_Prosthetic_Group_Biosynthesis__Biotin_Biosynthesis",
        "S_Cofactor_and_Prosthetic_Group_Biosynthesis___Pyrroloquinoline_quinone_biosynthesis",
        "S_Cofactor_and_Prosthetic_Group_Biosynthesis__Pantothenate_and_CoA_Biosynthesis",
        "S_Cofactor_and_Prosthetic_Group_Biosynthesis__Thiamine_Metabolism",
        "S_Cofactor_and_Prosthetic_Group_Biosynthesis__Porphyrin_and_Chlorophyll_Metabolism",
        "S_Cofactor_and_Prosthetic_Group_Biosynthesis__Riboflavin_Metabolism",
        "S_Cofactor_and_Prosthetic_Group_Biosynthesis__Queuosine_biosynthesis",
        "S_Cofactor_and_Prosthetic_Group_Biosynthesis__Folate_Biosynthesis",
        "S_Cofactor_and_Prosthetic_Group_Biosynthesis__One_Carbon_pool_by_folate"
    ],
    
    "Transport Systems": [
        "S_Transport__Inner_Membrane",
        "S_Transport__ABC_system",
        "S_Transport__Outer_Membrane",
        "S_Transport__solvent_extrusion",
        "Extracellular exchange"
    ],
    
    # "Energy Metabolism": [
    #     "S_Oxidative_Phosphorylation",
    #     "S_Butanoate_Metabolism",
    #     "S_Glyoxylate_and_dicarboxylate_metabolism"
    # ],
    
    "Aromatic Compound Degradation": [
        "S_Aromatic_Compounds_Degradation__Nicotinic_acid_pathway",
        "S_Aromatic_Compounds_Degradation__Toluene_Pathway",
        "S_Aromatic_Compounds_Degradation__Phenylacetyl_CoA_Catabolom",
        "S_Aromatic_Compounds_Degradation__Homogentisate_pathway",
        "S_Aromatic_Compounds_Degradation__B_Ketoadipate_pathway",
        "S_Aromatic_Compounds_Degradation__Gallic_acid_Metabolism"
    ],
    
    "Nitrogen Metabolism": [
        "S_Nitrogen_Metabolism",
        "S_Alternate_Carbon_and_Nitrogen_source__Ectoine_Metabolism",
        "S_Alternate_Carbon_and_Nitrogen_source__D_Amino_acids_Metabolism", 
        "S_Alternate_Carbon_and_Nitrogen_source__Hydroxy_proline_metabolism",
        "S_Alternate_Carbon_and_Nitrogen_source__Dipeptide_Metabolism",
        "S_Alternate_Carbon_and_Nitrogen_source__Amines_and_Polyamines_Metabolism"
    ],
    
    "Metal and Inorganic Metabolism": [
        "S_Heavy_Metal_Tolerance",
        "S_Iron_uptake_and_metabolism",
        "S_Sulfur_Metabolism",
        "S_Phosphonate_and_phosphinate_metabolism",
        "S_Inorganic_polyphosphates_metabolism"
    ],
    
    "Cellular Processes and Storage": [
        "S_PHAs_Metabolism",
        "S_Alginate_biosynthesis", 
        "S_tRNA_Charging",
        "S_Formaldehyde_Metabolism"
    ],
    
    "Stress Response and Environmental Adaptation": [
        "S_Xenobiotic_tolerance",
        "S_Plant_growth_promoting"
    ],
    
    "Core Cellular Functions": [
        "Biomass and maintenance functions",
        "Intracellular demand",
    ]
}


pathway_to_category = {}
for category, pathways in metabolic_categories.items():
    for pathway in pathways:
        pathway_to_category[pathway] = category

        

In [6]:
pathway_to_category

for rxn in old_model.reactions:
    if rxn.subsystem in list(pathway_to_category.keys()):
        rxn.subsystem = pathway_to_category[rxn.subsystem]
    elif(rxn.subsystem=='Unassigned'):
        rxn.subsystem = 'Uassigned'
    else:
        rxn.subsystem = 'Unassigned'

In [7]:
old_model.reactions[20].subsystem

'Fatty Acid and Lipid Metabolism'

In [11]:
for rxn in model.reactions:
    if(rxn.id in [w.id for w in old_model.reactions]):
        rxn.subsystem = old_model.reactions.get_by_id(rxn.id).subsystem
    elif('transport' in rxn.name.lower() or 'symport' in rxn.name.lower() or 'exchange' in rxn.name.lower()):
        rxn.subsystem = 'Transport Systems'
    else:
        rxn.subsystem = 'Unassigned'


        

In [12]:
rxn_by_subsystem = pd.DataFrame()
rxn_by_subsystem['ID'] = [rxn.id for rxn in model.reactions]
rxn_by_subsystem['Reaction'] = [rxn.reaction for rxn in model.reactions]
rxn_by_subsystem['Name'] = [rxn.name for rxn in model.reactions]
rxn_by_subsystem['Subsystem'] = [rxn.subsystem for rxn in model.reactions]


In [25]:
rxn_by_subsystem[rxn_by_subsystem['Subsystem']=='Unassigned'].to_csv('unassigned_reactions.csv', index=False)

In [27]:
gpt_subsystems = pd.read_csv('annotated_reactions_alt_dict.csv')
gpt_subsystems['Assigned_Top_Category'].value_counts()

Unclassified                           78
Amino Acid Metabolism                  73
Cofactor and Vitamin Biosynthesis      41
Central Carbon Metabolism              34
Transport Systems                      31
Alternative Carbon Metabolism          27
Nitrogen Metabolism                    17
Fatty Acid and Lipid Metabolism        15
Metal and Inorganic Metabolism         14
Cellular Processes and Storage         12
Nucleotide Metabolism                  11
Aromatic Compound Degradation           7
Cell Envelope and Wall Biosynthesis     2
Name: Assigned_Top_Category, dtype: int64

In [29]:
gpt_subsystems[gpt_subsystems['Assigned_Top_Category']=='Unclassified'].to_csv('gpt_unclassified_reactions.csv')